## Research commodity options calendar spreads, especially in ES
### Try to optimize rolling portfolio protection

## IF YOU WANT TO SEE WARNINGS, COMMENT THIS OUT

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb,pdb
import traceback
import pandas_datareader.data as pdr
from scipy.stats import norm

from ipysheet import from_dataframe,to_dataframe
# importlib.reload(build_db)

In [ ]:
dir(mibian.GK)

### important global variables

In [ ]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'
SYMBOL_TO_RESEARCH = 'ES'
STRIKE_DIVISORS = {}
USE_PYVOL = True

# df_expiry_dates_additions = pd.read_csv('df_expiry_dates_additions.csv')
df_expiry_dates_additions = pd.read_csv('live_option_expirations.csv')


#### get all contracts in the options database

In [ ]:
pga = db_info.get_db_info()
print(f"futtab max date: {pga.get_sql(f'select max(settle_date) from {futtab}')}")
print(f"opttab max date: {pga.get_sql(f'select max(settle_date) from {opttab}')}")


In [ ]:
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,width=800,height=400,
                number_of_ticks_display=20,
                yaxis2_cols=None,
                x_value_labels=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    tdtext = tdvals
    if x_value_labels is not None:
        tdtext = [x_value_labels[i] for i in tdvals]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdtext,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        autosize=False,
        width=width,
        height=height,
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    fig.update_layout(
        title={
            'text': plot_title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    return fig

def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
    ld_shapes = []
    for beg_end_date_tuple in beg_end_date_tuple_list:
        ld_beg = beg_end_date_tuple[0]
        ld_end = beg_end_date_tuple[1]
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=ld_beg[i],
            y0=0,
            x1=ld_end[i],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    return fig

In [ ]:
def _next_monthyear_code(contract):
    code_val = contract[-3]
    code_num = DICT_MONTH_NUMS[code_val]
    y = int(contract[-2:])
    if code_num+1>12:
        next_code_num = 1
        next_y = y + 1
    else:
        next_code_num = code_num+1
        next_y = y
    next_code_val = MONTH_CODES[next_code_num-1]
    next_contract = contract[0:-3] + next_code_val + '%02d' %(next_y)
    return next_contract

def get_postgres_data(contract,strike_divisor=None):
    '''
    Get options and underlying data for ONLY ONE CONTRACT
    '''
    osql = f"select * from {opttab} where symbol='{contract}';"
    dfo = pga.get_sql(osql)
    if len(dfo)<10:
        e = f'''
        get_postgres_data ERROR: not enough option data for contract {contract} 
        '''
        raise ValueError(e)
    num_settle_days = len(dfo.settle_date.unique())
    u_contract = contract
    for i in range(12):
        usql = f"select * from {futtab} where symbol='{u_contract}';"
        dfu = pga.get_sql(usql)
        if len(dfu) < num_settle_days:
            u_contract = _next_monthyear_code(u_contract)
            print(f'trying contract {u_contract}')
        else:
            break

    if len(dfu)< num_settle_days:
        e = f'''
        get_postgres_data ERROR: not enough underlying days found for options contract {contract} 
        where len(underlying) = {len(dfu)} and num_settle_days = {num_settle_days}
        '''
        raise ValueError(e)
    # Merge options and futures data
    dfu = dfu.rename(columns={'symbol':'u_symbol'})
    df = dfo.merge(dfu,how='inner',on=['settle_date'])
    # Get options expiration dates
    df_expiry_dates = dfo[['symbol','settle_date']].groupby('symbol',as_index=False).max()
    df_additions = df_expiry_dates_additions[df_expiry_dates_additions.symbol==contract]
    df_additions = df_additions[['symbol','yyyymmdd_option']].rename(columns={'yyyymmdd_option':'settle_date'})
    additional_symbols = df_additions.symbol.values
    df_expiry_dates = df_expiry_dates[~df_expiry_dates.symbol.isin(additional_symbols)]
    df_expiry_dates = df_expiry_dates.append(df_additions).sort_values('symbol').copy()
    if strike_divisor is not None:
        df.strike = df.strike/strike_divisor
    return df,df_expiry_dates

### Methods to build implied volatilites

In [ ]:
def _get_contract_number_from_symbol(symbol):
    c = symbol[0:2]
    if c in ['CL','CB','ES','GE','NG']:
        return 2
    return 2

In [ ]:
def lam_pyvol(r):
    try:
        return implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
    except:
        return -1
# lam_pyvol = lambda r:implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
lam_mibian = lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility

def get_implieds(df,df_expiry_dates,contract,contract_num=2):
    df2 = df[['symbol','contract_num','pc','settle_date','strike','close_x','close_y']]
    df2 = df2[(((df2.pc=='C' )& (df2.strike>=df2.close_y)) | ((df2.pc=='P' ) & (df2.strike<df2.close_y)))  & (df2.symbol.str.contains(contract))]
    cnum = _get_contract_number_from_symbol(contract)
    if contract_num is not None:
        df2 = df2[df2.contract_num==contract_num]
    if len(df2)<1:
        raise ValueError(f"get_implieds: no contracts = contract_num {contract_num}")
    phigh = df2.close_y.max()
    plow = df2.close_y.min()
    high_strike = round(phigh * 1.3)
    low_strike = round(plow * .7)
    df2 = df2[(df2.strike>=low_strike) & (df2.strike<=high_strike)]

    df9 = df2[df2.symbol==contract]
    df9 = df9.merge(df_expiry_dates.rename(columns={'settle_date':'expiry'}),on='symbol',how='inner')
    df9['syear'] = df9.settle_date.astype(str).str.slice(0,4).astype(int)
    df9['smon'] = df9.settle_date.astype(str).str.slice(4,6).astype(int)
    df9['sday'] = df9.settle_date.astype(str).str.slice(6,8).astype(int)
    df9['eyear'] = df9.expiry.astype(str).str.slice(0,4).astype(int)
    df9['emon'] = df9.expiry.astype(str).str.slice(4,6).astype(int)
    df9['eday'] = df9.expiry.astype(str).str.slice(6,8).astype(int)
    df9['sdatetime'] = df9.apply(lambda r:datetime.datetime(r.syear,r.smon,r.sday),axis=1)
    df9['edatetime'] = df9.apply(lambda r:datetime.datetime(r.eyear,r.emon,r.eday),axis=1)
    df9['dte'] = df9.edatetime - df9.sdatetime
    df9.dte = df9.dte.dt.days
    df9 = df9[['symbol','settle_date','pc','contract_num','strike','close_x','close_y','dte']]
    df10 = df9.iloc[:len(df9)].copy()
    df10.index = list(range(len(df10)))
    if USE_PYVOL:
        df10['iv'] = df10.apply(lam_pyvol,axis=1)
    else:
        n = 100
        for i in tqdm_notebook(np.arange(0,len(df10)-n,n)):
                df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lam_mibian,axis=1)
        print(f'doing remaining {datetime.datetime.now()}')
        i = df10[df10.iv.isna()].index[0]
        df10.loc[i:,'iv'] = df10.loc[i:].apply(lam_mbian,axis=1)
        print(f'done with remaining {datetime.datetime.now()}')
    return df10



#### example of using mibian for options calcs (we use py_vollib instead)

In [ ]:
def _test_mibian():
    underlying=1.4565
    strike=1.45
    interest = 1
    days=30
    opt_info = [underlying,strike,interest,days]
    c = mibian.BS(opt_info, volatility=20)
    print(c.callPrice,
    c.putPrice,
    c.callDelta,
    c.putDelta,
    c.callDelta2,
    c.putDelta2,
    c.callTheta,
    c.putTheta,
    c.callRho,
    c.putRho,
    c.vega,
    c.gamma)


    co = mibian.BS(opt_info, callPrice=c.callPrice)
    co.impliedVolatility

In [ ]:
_test_mibian()

#### Show simple example of using py_vol package

In [ ]:
def _test_py_vollib():
    #CL,Q2019,560P,07/02/2019,0.6,1.61,0.54,1.54,1997,4465
    F = 56.25
    K = 56
    sigma = .366591539
    flag = 'p'
    t = 15/365.0
    r = .025
    discounted_call_price = black.black(flag, F, K, t, r, sigma)
    dcp = 1.54
    ivpy = implied_volatility.implied_volatility(dcp, F, K, r, t, flag)
    ivmn = mibian.BS([F,K,2.5,15], callPrice=dcp).impliedVolatility
    return discounted_call_price,ivpy,ivmn


In [ ]:
_test_py_vollib()

### Method to get closest-to-the-money and at-the-money (atm) vols

In [ ]:
def get_closest_to_the_money_vols(df_implied_vols,
                 opt_close_col='close_x',
                 fut_close_col = 'close_y',
                 iv_col = 'iv'):
    df_in = df_implied_vols.copy()
    df_in['amt_away_from_money'] = df_in.apply(lambda r: abs(r[fut_close_col]-r.strike),axis=1)
    gb_cols = ['settle_date','symbol']
    df_min_amt_away = df_in[gb_cols + ['amt_away_from_money']].groupby(gb_cols,as_index=False).min()
    df_ret = df_min_amt_away.merge(
        df_in,on=gb_cols + ['amt_away_from_money'],how='inner')
    df_lowest_iv = df_ret[['settle_date','symbol','iv']].groupby(
        ['settle_date','symbol'],
        as_index=False).min()
    df_ret = df_ret.merge(df_lowest_iv,on=['settle_date','symbol','iv'],how='inner')
    return df_ret


In [ ]:
def get_even_moneyness_strikes(df_implieds,low_range=-.3,high_range=.4,range_increment=.05,rounding=6):
    df10 = df_implieds.copy()
    # define amounts around the money which will help create strikes to add
    lr = -6*range_increment
    hr = 8*range_increment
#     low_perc = 1 + low_range
#     high_perc = 1 + high_range
    low_perc = 1 + lr
    high_perc = 1 + hr
#     moneyness = np.arange(.7,1.4,.05).round(rounding)
    moneyness = np.arange(low_perc,high_perc,range_increment).round(rounding)
    # define columns on which to execute groupby
    gb_cols = ['symbol','settle_date','contract_num','dte','close_y']
    # define function used in groupby.apply to create strikes and iv's at those strikes
    #   where the strikes are an even amount from the money 
    #   (like .7, .8, ... 1, 1.1, 1.2, etc)
    def _add_even_moneyness_strikes(df):
        # get underlying from first row (the groupby makes them all the same)
        r = df.iloc[0]
        underlying = r.close_y
        # create new rows to append to df, using only the gb_cols
        df_ret1 = df.iloc[:len(moneyness)][gb_cols].copy()
        if len(df_ret1)<len(moneyness):
            return None
        # add nan iv's !!!! MUST BE np.nan - NOT None
        df_ret1['iv'] = np.nan
        # add new strikes
        df_ret1['strike'] = moneyness * underlying
        # append the new strikes
        try:
            # try with using the sort=True options for versions of pandas after 0.23
            dfa = df.append(df_ret1,ignore_index=True,sort=True).copy()
        except:
            # otherwise do not specify sort
            dfa = df.append(df_ret1,ignore_index=True).copy()
        df_ret2 = dfa.sort_values(['symbol','settle_date','pc','strike'])
        df_ret2 = df_ret2.drop_duplicates(subset='strike')
        # set the index to the strike so that interpolate works
        df_ret2.index = df_ret2.strike
        # create interpolated iv's
        df_ret2['iv'] = df_ret2.iv.interpolate(method='polynomial', order=2)
        # reset the index
        df_ret2.index = list(range(len(df_ret2)))
        return df_ret2

    # start here
    df11 = df10.groupby(gb_cols).apply(_add_even_moneyness_strikes).copy()
    df11.index = list(range(len(df11)))
    df11['moneyness'] = df11.strike / df11.close_y
    df11.moneyness = df11.moneyness.round(rounding)

    df12 = df11[(df11.moneyness.isin(moneyness)) & (~df11.iv.isna())].copy()
    df12.moneyness  = df12.moneyness - 1
    df12.index = list(range(len(df12)))
    df12_atm = df12[df12.moneyness.round(rounding)==0][['symbol','settle_date','pc','iv']]

    df12_atm = df12_atm.rename(columns={'iv':'atm_iv'})
    
    df12_atm = df12_atm.drop_duplicates()
    df12_atm.pc = ''#np.nan
    df12.pc = ''#np.nan
    df12 = df12.merge(df12_atm,on=['symbol','settle_date','pc'],how='inner')

#     df12 = df12.merge(df12_atm,on=['symbol','settle_date'],how='inner')
#     df12['pc'] = np.nan

    df12.moneyness = df12.moneyness.round(rounding)
    df12['vol_skew'] = (df12.iv - df12.atm_iv).round(rounding)
    # now get rid of cases where an even money strike, and an actual strike are the same, so that you have 
    #    2 vol_skew records for the same 'settle_date','symbol','moneyness' combination
    # THIS IS THE EASY WAY
#     df12 = df12[df12.close_x.isna()]
    # THIS IS THE HARD WAY
    gb_cols = ['settle_date','symbol','moneyness','close_y','contract_num','dte']
    val_cols = ['vol_skew','iv','strike','atm_iv']
    df12 = df12[gb_cols + val_cols].groupby(gb_cols,as_index=False).mean()

    return df12



### Method to get volatility skews per symbol

In [ ]:
def create_skew_per_date_df(df,rounding=6):
    '''
    Find the first settle_date whose count of rows is equal to max count of rows.
    '''
    # get the first symbol (which should be the only symbol)
    contract = df.symbol.unique()[0]
    # get just that symbol's data
    df12 = df[df.symbol==contract]
    df_counts = df12[['settle_date','moneyness']].groupby('settle_date',as_index=False).count()
    max_count = df_counts.moneyness.max()
    first_max_count_settle_date = df_counts[df_counts.moneyness==max_count].iloc[0].settle_date
    
    df_ret = df12[df12.settle_date==first_max_count_settle_date][['moneyness']]
    all_settle_dates = sorted(df_counts.settle_date.unique())
    for settle_date in all_settle_dates:
        df_temp = df12[df12.settle_date==settle_date][['moneyness','vol_skew']]
        df_ret = df_ret.merge(df_temp,on='moneyness',how='outer')
        df_ret = df_ret.rename(columns={'vol_skew':str(settle_date)})
    df_ret = df_ret.sort_values('moneyness')
    df_ret.moneyness = df_ret.moneyness.round(rounding)
    return df_ret


def skew_per_symbol(symbol,strike_divisor=None,contract_num=2,
                   low_range=-.3,high_range=.4,
                   range_increment=.05):
    '''
    For a symbol like CLM16 or EZH19, create 2 Dataframes
      1. df_iv - contains rows of implied vols, for only the 'pseudo' strikes that are an even
                 percent away from the money for each settle_date
      2. df_skew - contains one row per day of skew data of for 'pseudo' strikes that are an even
                 percent away from the money for each settle_date
    '''
    _exception = None
    _stacktrace = None
    df_iv = None
    df_skew = None
    df,df_expiry_dates = get_postgres_data(symbol)
    if len(df[df.contract_num==contract_num])>0:
        df10 = get_implieds(df,df_expiry_dates,symbol,contract_num)
        df12 = get_even_moneyness_strikes(df10,low_range=low_moneyness_range,
                                         high_range=high_range,
                                         range_increment=range_increment)
        df_sk = create_skew_per_date_df(df12)
        df_sk.index = list(range(len(df_sk)))
        df_skt = df_sk.T
        df_skt.columns = df_skt.loc['moneyness']
        df_skt = df_skt.iloc[1:].copy()
        df_skt['symbol'] = symbol
        df_skt['settle_date'] = df_skt.index
        df_iv = df12.copy() 
        df_skew = df_skt.copy()
        df_skew = df_skew.sort_values('settle_date')
        df_skew.index = list(range(len(df22)))

    return df_iv,df_skew

### test getting implieds for a single contract

In [ ]:
def get_all_implied_info(symbol,contract_num = 2,
                            low_range=-.3,high_range=.4,
                            range_increment=.05,
                            rounding=6,
                            postgres_data_tuple=None):
    '''
    return a dictionary of historical dataframes:
    {'hist_data':df,'implieds':df_implieds,'even_money':df_even_money,'vols':df_vols}
    '''
    df_implieds = None
    df_even_money = None
    df_vols = None
    ret = {'implieds':df_implieds,'even_money':df_even_money,'vols':df_vols}
    commod = symbol[0:2]
    strike_div = None if commod not in STRIKE_DIVISORS.keys() else STRIKE_DIVISORS[commod]
    
    if postgres_data_tuple is None:
        df,df_expiry_dates = get_postgres_data(symbol)
    else:
        df,df_expiry_dates = postgres_data_tuple
    if df is None or len(df)<10:
        print(f'no data for symbol: {symbol}')
        return {'implieds':None,'even_money':None,'vols':None}
    df_implieds = get_implieds(df,df_expiry_dates,symbol,contract_num=contract_num)
    if df_implieds is not None:
        df_ctm_vols = get_closest_to_the_money_vols(df_implieds)
        df_even_money = get_even_moneyness_strikes(
                            df_implieds,low_range=low_range,
                            high_range=high_range,range_increment=range_increment,
                            rounding=rounding)
        df_atm_money = df_even_money[df_even_money.moneyness==0]
        df_vols = df_ctm_vols.merge(
            df_atm_money[['settle_date','symbol','atm_iv']],
            on=['settle_date','symbol'],how='inner')
    return {'hist_data':df,'implieds':df_implieds,'even_money':df_even_money,'vols':df_vols}




#### test get_all_implied_info

In [ ]:
all_implied_test = get_all_implied_info('CLZ20',contract_num=2,range_increment=.036)
all_implied_test['even_money']

### Options strategy history

In [ ]:
def ot_from_sn(option_short_name):
    '''
    get OptTrade instance from option_short_name string like 'ESM20_2500_C_-4'
    YOU must have at least 'ESM20_2500_C'.  If no quantity, it will be zero.
    '''
    if option_short_name is None:
        return None
    parts = option_short_name.split('_')
    if len(parts)<3:
        return None
    sym = parts[0]
    strike = parts[1]
    pc = parts[2]
    qty = 0 if len(parts)<4 else float(str(parts[3]))
    return OptTrade(sym,strike,pc,qty)

class OptTrade():
    def __init__(self,symbol,strike,pc,qty,pga=None):
        self.symbol = symbol
        self.strike=strike
        self.pc = pc.upper()
        self. qty = qty
    def __str__(self):
        return f"{self.symbol}_{self.strike}_{self.pc}_{self.qty}"   
    def get_postgres_data(self,rounding=6):
        df,df_expiry = get_postgres_data(self.symbol)
        if df is None or len(df)<1:
            return None
        df = df[df.pc.str.lower()==self.pc.lower()]
        df = df[df.strike.astype(float).round(rounding)==round(float(str(self.strike)),rounding)]
        df.index = list(range(len(df)))
        return df
    def get_implied_info(self,contract_num=2,
                            low_range=-.3,high_range=.4,
                            range_increment=.05,rounding=6):
        '''
        return:
        {'hist_data':df,'implieds':df_implieds,'even_money':df_even_money,'vols':df_vols}
        '''
        return get_all_implied_info(self.symbol,contract_num=contract_num,
                                   low_range=low_range,high_range=high_range,
                                   range_increment=range_increment,rounding=rounding)


def _black(r):
    flag = 'p' if r.moneyness < 0 else 'c'
    F = r.close_y
    K = r.strike
    t = r.dte/365
    rate = .015
    sigma = r.atm_iv + r.vol_skew
    return black.black(flag, F, K, t, rate, sigma)

def _delta(r,rate=.015):
    flag = 'p' if r.moneyness < 0 else 'c'
    F = r.close_y
    K = r.strike
    t = r.dte/365
    sigma = r.atm_iv + r.vol_skew
    b =  black.black(flag, F, K, t, rate, sigma)
    bdn =  black.black(flag, F*.9975, K, t, rate, sigma)
    bup =  black.black(flag, F, K*1.0025, t, rate, sigma)
    delta_up = abs((bup-b)/(F*.0025))
    delta_dn = abs((bdn-b)/(F*.0025))
    return (delta_up+delta_dn)/2

class MultiLeg():
    '''
    create a history of a multi-leg option
    '''
    opt_cols = ['strike','pc','open_x','high_x','low_x','close_x',
              'adj_close_x','volume_x','open_interest_x']
    fut_cols = ['open_y','high_y','low_y','close_y',
              'adj_close_y','volume_y','open_interest_y']
    both_cols = ['settle_date','symbol']
    def __init__(self,leg_list: list,contract_num_list=None,
                low_range=-.3,high_range=.4,range_increment=.05):
        self.leg_list = leg_list
        self.contract_num_list = [None for _ in range(len(leg_list))] if contract_num_list is None else contract_num_list
        self.low_range = low_range
        self.high_range = high_range
        self.range_increment = range_increment
        # get history for each leg
        hist_list = []
        for i in range(len(leg_list)):
            leg = leg_list[i]
            contract_num = self.contract_num_list[i]
            print(f"MultiLeg geting data for leg {str(leg)} ")
            hist_dict = leg.get_implied_info(contract_num=contract_num,
                            low_range=low_range,high_range=high_range,
                                             range_increment=range_increment)
            hist_list.append(hist_dict)
        self.hist_list = hist_list

        
        
    def get_values(self,column_list=None):
        col_list = ['close_x'] if column_list is None else column_list
        yyyymmdd_set = None

        for i in range(len(self.hist_list)):
            df = self.hist_list[i]['hist_data']
            yyyymmdd_set_df = set(df.settle_date.values)
            if yyyymmdd_set is None:
                yyyymmdd_set = yyyymmdd_set_df
            else:
                if len(yyyymmdd_set)>len(yyyymmdd_set_df):
                    yyyymmdd_set = yyyymmdd_set.intersection(yyyymmdd_set_df)
                else:
                    yyyymmdd_set = yyyymmdd_set_df.intersection(yyyymmdd_set)
            # save all data for this eg
        yyyymmdd_set = list(yyyymmdd_set)
        hist_df_list = []
        for i in range(len(self.leg_list)):
            leg = self.leg_list[i]
            df = self.hist_list[i]['hist_data'].copy()
            df['short_name'] = str(leg)
            df = df[(df.strike==float(str(leg.strike))) & (df.pc.str.upper()==leg.pc.upper())]
            df = df[df.settle_date.isin(yyyymmdd_set)]
            df.index = list(range(len(df)))
            hist_df_list.append(df)

        df = hist_df_list[0][['settle_date']]
        for c in col_list:
            c_values = np.zeros(len(df))
            for i in range(len(self.leg_list)):
                leg = self.leg_list[i]
                qty = float(leg.qty)
                df_hist = hist_df_list[i].copy()
                df_hist[c].fillna(0)
                c_values = c_values + df_hist[c].values * qty
            df[c] = c_values
        
        return df
    
    def get_theo_values(self):
        df_all_legs = None
        both_merge_cols = ['settle_date','moneyness']
        # get the even_money dataframe for each leg, and get theo prices from it
        for i in range(len(self.hist_list)):
            # get the dictionary of returned history values
            hist_dict = self.hist_list[i]
            # get the dataframe with the even money strike values
            df_even_money = hist_dict['even_money'].copy()
            # create a list of price cols for each leg
            optprice_col = f"optprice_leg_{i}"
            # get a black scholes option price for each even money strike
            df_even_money[optprice_col] = df_even_money.apply(_black,axis=1)
            # limit the columns to the merge cols and the optprice col for this leg
            df_even_money = df_even_money[both_merge_cols +[optprice_col]]
            # merge all the leg option values into one dataframe
            if df_all_legs is None:
                df_all_legs = df_even_money.copy()
            else:
                df_all_legs = df_all_legs.merge(df_even_money,on=both_merge_cols,how='inner')

        # now combine the optprices
        op_cols = [f"optprice_leg_{i}" for i in range(len(self.hist_list))]
        df_all_legs['strat_price'] = 0
        for i in range(len(op_cols)):
            c = op_cols[i]
            leg = self.leg_list[i]
            qty = leg.qty
            df_all_legs.strat_price = df_all_legs.strat_price + df_all_legs[c] * qty

        return df_all_legs
                    
                    
        


### Graph atm_vol vs price for multiple contracts

In [ ]:
begin_year = 20
commod = 'ES'
sql = f"""
select distinct symbol from {opttab} where substring(symbol,1,2)='{commod}'
and substring(symbol,4,2)::int >= {begin_year}
"""
all_contracts = pga.get_sql(sql).symbol.unique()
ac_sorted = sorted([c[0:2]+c[3:]+c[2] for c in all_contracts])
all_contracts = [c[0:2]+c[-1]+c[2:4] for c in ac_sorted]
for sym in [c for c in all_contracts if int(c[-2:])>=begin_year]:
    print(f"graphing symbol: {sym}")
#     commod = sym[0:2]
    strike_div = None if commod not in STRIKE_DIVISORS.keys() else STRIKE_DIVISORS[commod]
    try:
        d = get_all_implied_info(sym,contract_num=None)
    except Exception as e:
        print(f'Exception getting implied vols for symbol: {sym}')
        print(str(e))
        import traceback
        traceback.print_exc()
        continue
    df_implieds = d['implieds']
    if df_implieds is None:
        print(f'no implied vols for symbol: {sym}')
        continue
    df_vols = d['vols']
    
    iplot(plotly_plot(
        df_in=df_vols[['settle_date','close_y','atm_iv']],
        x_column='settle_date',yaxis2_cols=['atm_iv'],number_of_ticks_display=15)
         )    

### show some examples of ```OptTrade``` and ```MultiLeg```


In [ ]:
legs = ['ESM20_2500_P_-1','ESZ20_2500_P_1']
ot_legs = [ot_from_sn(leg) for leg in legs]
[str(ot_leg) for ot_leg in ot_legs]


df_leg0_even_money = ot_legs[0].get_implied_info(contract_num=None)['even_money']
df_leg1_even_money = ot_legs[1].get_implied_info(contract_num=None)['even_money']


In [ ]:
df_leg0_even_money

### Examples of ```black.black```

In [ ]:
black.black('p', 100, 75, 1, .02, .15)

In [ ]:
def _black(r):
    flag = 'p' if r.moneyness < 0 else 'c'
    F = r.close_y
    K = r.strike
    t = r.dte/365
    rate = .015
    sigma = r.atm_iv + r.vol_skew
    return black.black(flag, F, K, t, rate, sigma)
df_leg0_even_money2 = df_leg0_even_money.copy()
df_leg0_even_money2['optprice_leg0'] = df_leg0_even_money2.apply(_black,axis=1)

df_leg1_even_money2 = df_leg1_even_money.copy()
df_leg1_even_money2['optprice_leg1'] = df_leg1_even_money2.apply(_black,axis=1)
both_merge_cols = ['settle_date','moneyness']
df_leg0_even_money2 = df_leg0_even_money2[both_merge_cols + ['optprice_leg0']]
df_leg1_even_money2 = df_leg1_even_money2[both_merge_cols + ['optprice_leg1']]
df_both_legs = df_leg0_even_money2.merge(df_leg1_even_money2,on=both_merge_cols,how='inner')
df_both_legs['optprice_spread'] = df_both_legs.optprice_leg1 - df_both_legs.optprice_leg0
df_both_legs[df_both_legs.moneyness==-.15]



In [ ]:
legs = ['ESM20_2500_P_-1','ESZ20_2500_P_1']
ot_legs = [ot_from_sn(leg) for leg in legs]
es_m0z0_2500 = MultiLeg(ot_legs)

In [ ]:
es_m0z0_2500.get_values(['close_x','open_x','high_x','low_x'])

In [ ]:
df_es_m0z0_2500_theo_values  = es_m0z0_2500.get_theo_values()
for n in [0.0,-0.1,-0.2,-0.3]:
    display.display(df_es_m0z0_2500_theo_values[df_es_m0z0_2500_theo_values.moneyness==n].tail())
for n in [0.1,0.2,0.3]:
    display.display(df_es_m0z0_2500_theo_values[df_es_m0z0_2500_theo_values.moneyness==n].tail())    

### show historical conversion values

In [ ]:
df_m20 = es_m0z0_2500.hist_list[0]['hist_data']

In [ ]:
strike = 2750
df_m20_calls = df_m20[(df_m20.strike== strike) & (df_m20.pc== 'C')][['settle_date','strike','close_x','close_y']]
df_m20_calls = df_m20_calls.rename(columns={'close_x':'callprice'})
df_m20_puts = df_m20[(df_m20.strike== strike) & (df_m20.pc== 'P')][['settle_date','strike','close_x']]
df_m20_puts = df_m20_puts.rename(columns={'close_x':'putprice'})
df_m20_both = df_m20_calls.merge(df_m20_puts,on=['settle_date','strike'],how='inner')
df_m20_both['conversion'] = df_m20_both.callprice + df_m20_both.strike - (df_m20_both.putprice + df_m20_both.close_y)
iplot(plotly_plot(df_in=df_m20_both[['settle_date','conversion','close_y']],x_column='settle_date',
                 yaxis2_cols=['close_y']))
display.display(df_m20_both)



### Show a calendar spread stragegy

In [ ]:
legs = ['ESM20_2750_P_-1','ESZ20_2750_P_1']
ot_legs = [ot_from_sn(leg) for leg in legs]
es_m0z0_2750 = MultiLeg(ot_legs)

In [ ]:
es_m0z0_2750.get_values(['close_x'])

### Show a 1X2 ES put spread

In [ ]:
legs = ['ESZ20_2400_P_1.0','ESZ20_1800_P_-2.0']
ot_legs = [ot_from_sn(leg) for leg in legs]

ml = MultiLeg(ot_legs)
mltv = ml.get_values()


In [ ]:
mltv

In [ ]:
df_ml_tv = ml.get_theo_values()
df_ml_tv

In [ ]:
all_info = get_all_implied_info('ESZ20',contract_num=4)

In [ ]:
all_info.keys()

In [ ]:
df_even_money = all_info['even_money'][['settle_date','moneyness','dte','atm_iv','vol_skew']]
df_even_money = df_even_money[df_even_money.moneyness==-.1]
df_even_money

### Calculate hedge ratios for S&P positions



In [ ]:
es_multiplier = 50
curr_es = 3000
curr_posvalue = 400000 #stock dollar amount
curr_sp_equivilent = curr_posvalue / (curr_es*es_multiplier)
acceptable_loss = .125
hedge_strike = curr_es * (1-acceptable_loss)
hedge_posvalue = hedge_strike * curr_sp_equivilent * es_multiplier
curr_posvalue,curr_sp_equivilent,hedge_strike,hedge_posvalue

In [ ]:
black.black('c', 100, 100, 1, .017, .13)


In [ ]:
df_esz99 = pga.get_sql(f"select * from {futtab} where symbol='ESZ99'")

In [ ]:
years_per_graph = 2
for y in np.arange(10,21,years_per_graph):
    yyyymmdd_low = (2000+y)*100*100 + 101
    yyyymmdd_high = (2000+y+years_per_graph)*100*100 + 101
    c1 = df_esz99.settle_date>=yyyymmdd_low
    c2 = df_esz99.settle_date<=yyyymmdd_high
    dft = df_esz99[c1 & c2][['settle_date','close']]
    iplot(plotly_plot(df_in=dft,x_column='settle_date'))

In [ ]:
threshold = 1.125
dft = df_esz99[['settle_date','close','high','low']]
dft['current_high'] = dft.high.expanding(min_periods=1).max()
curr_10 = dft.iloc[0].current_high
curr_10_array = [curr_10]
for i in range(1,len(dft)):
    next_divby_curr = dft.iloc[i].current_high / curr_10
    if next_divby_curr >=threshold:
        curr_10 = dft.iloc[i].current_high
    curr_10_array.append(curr_10)
dft['curr_10'] = curr_10_array

In [ ]:
iplot(plotly_plot(df_in=dft[['settle_date','close','current_high','curr_10']],x_column='settle_date'))

In [ ]:
rnd = 6
inc = round(200/2775,rnd)
leg = ot_from_sn('ESM20_2500_P_1')
leg_data = leg.get_implied_info(contract_num=1,range_increment=inc,rounding=rnd)
df_l = leg_data['even_money']
df_lc = df_l[df_l.settle_date==df_l.settle_date.max()]


In [ ]:
close = df_lc.close_y.max()
rup = df_lc[df_lc.moneyness==inc].iloc[0]
vup = rup.iv
sup = rup.strike
c = black.black('c', close, sup, rup.dte/365, .01, vup)
rdn = df_lc[df_lc.moneyness==-inc].iloc[0]
vdn = rdn.iv
sdn = rdn.strike
p = black.black('p', close, sdn, rdn.dte/365, .01, vdn)
cdn = black.black('c', close, sdn, rdn.dte/365, .01, vdn)
{'close':close,'sup':sup,'vup':vup,'sdn':sdn,'vdn':vdn,
 'c':c,'p':p,'cdn':cdn,'p-c':p-c,
 'sup-sdn':sup-sdn,'gain_dn':cdn-c,'loss_up':sup-sdn-(cdn-c)}

In [ ]:
df_lc['delta'] = df_lc.apply(_delta,axis=1)
df_lc['price'] = df_lc.apply(_black,axis=1)

In [ ]:
df_lc

## END